# SMEFiT Tutorial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LHCfitNikhef/smefit_release/blob/tutorial/tutorial/tutorial.ipynb)

In [1]:
import sys
import os
import wget
import subprocess
import pathlib
print("Python version:", sys.version)

Python version: 3.11.8 (main, Feb 26 2024, 15:43:17) [Clang 14.0.6 ]


In [19]:
import smefit
import smefit.runner
import smefit.log as log

# setup log file
# smefit.log.setup_console(pathlib.Path("./run.log"))
log.setup_console("./output.txt")

In [23]:
# import logging 
# logging.FileHandler('my_log_file.log')

<FileHandler /Users/jaco/Documents/smefit_release/tutorial/my_log_file.log (NOTSET)>

In [24]:
# log.console.log("Running : analytic solution.")

[15:34:53] Running : analytic solution.                                                             ]8;id=2752;file:///var/folders/nj/zqktg5zx7nj88cnf_ljdhpbh0000gn/T/ipykernel_18092/3138680030.py\3138680030.py]8;;\:]8;id=754567;file:///var/folders/nj/zqktg5zx7nj88cnf_ljdhpbh0000gn/T/ipykernel_18092/3138680030.py#1\1]8;;\

In [3]:
def file_downloader(url, download_dir='./downloads'):

    if not os.path.exists(download_dir):
        os.mkdir(download_dir)

    file = wget.download(url, out=download_dir)
    return file

In [4]:
smefit_database_url = 'https://github.com/LHCfitNikhef/smefit_database/archive/refs/heads/main.zip'
smefit_database = file_downloader(smefit_database_url);

-1 / unknown

In [5]:
result = subprocess.run([f"unzip {smefit_database} -d ./downloads"], capture_output=True, shell=True, text=True)

In [12]:
test_runner = smefit.runner.Runner.from_file(pathlib.Path("./tutorial.yaml"))
test_runner.global_analysis("A")

[15:20:05] WARNING  results/LHC_NLO_LIN_GLOB already found, overwriting old results                    ]8;id=768174;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/runner.py\runner.py]8;;\:]8;id=71209;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/runner.py#70\70]8;;\

           INFO     Loading dataset : ATLAS_CMS_SSinc_RunI                                            ]8;id=105;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py\loader.py]8;;\:]8;id=229197;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py#135\135]8;;\

           INFO     Loading dataset : ATLAS_CMS_tt_AC_8TeV                                            ]8;id=444711;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py\loader.py]8;;\:]8;id=396157;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py#135\135]8;;\

           INFO     Loading dataset : ATLAS_SSinc_RunII                                               ]8;id=610534;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py\loader.py]8;;\:]8;id=984336;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py#135\135]8;;\

           INFO     Loading dataset : ATLAS_STXS_runII_13TeV                                          ]8;id=402376;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py\loader.py]8;;\:]8;id=98868;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py#135\135]8;;\

...

           INFO     Loading dataset : LEP_eeWW_206GeV                                                 ]8;id=508895;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py\loader.py]8;;\:]8;id=241697;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py#135\135]8;;\

/Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/coefficients.py:216: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self._table.iloc[self.is_free, 0] = value


           INFO     Computing Analytic solution ...                                                 ]8;id=724520;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/optimize/analytic.py\analytic.py]8;;\:]8;id=304685;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/optimize/analytic.py#127\127]8;;\

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Parameter ┃ Best value ┃ Error ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ O11qq     │ 3.572      │ 3.785 │
│ O13qq     │ -0.080     │ 0.113 │
│ O1dt      │ -13.149    │ 9.306 │
│ O1qd      │ 9.267      │ 8.850 │
│ O1qt      │ 0.232      │ 2.656 │
│ O1qu      │ -4.066     │ 3.716 │
│ O1ut      │ 0.436      │ 6.338 │
│ O3pQ3     │ -0.244     │ 0.464 │
│ O3pl1     │ -0.035     │ 0.051 │
│ O3pl2     │ -0.028     │ 0.060 │
│ O3pl3     │ -0.041     │ 0.092 │
│ O3pq      │ -0.074     │ 0.037 │
│ O81qq     │ 0.832      │ 1.213 │
│ O83qq     │ -0.497     │ 2.438 │
│ O8dt      │ 4.190      │ 6.353 │
│ O8qd      │ -3.331     │ 5.479 │
│ O8qt      │ -0.214     │ 1.568 │
│ O8qu      │ 1.292      │ 4.292 │
│ O8ut      │ -2.683     │ 3.172 │
│ OWWW      │ 0.015      │ 0.300 │
│ Obp       │ 0.023      │ 0.033 │
│ Ocp       │ -0.043     │ 0.101 │
│ Oll       │ -0.006     │ 0.054 │
│ OpB       │ 0.130      │ 0.223 │
│ OpD       │ 0.044      │ 0.570 │
│ OpG       │ -0.007     │ 0.008 │
│ OpQM      │ 0.396      │ 0.927 │
│ OpW       │ 0.212      │ 0.250 │
│ OpWB      │ -0.010     │ 0.265 │
│ Opd       │ 0.098      │ 0.919 │
│ Opdi      │ -0.504     │ 0.270 │
│ Ope       │ -0.028     │ 0.286 │
│ Opl1      │ -0.002     │ 0.141 │
│ Opl2      │ -0.015     │ 0.164 │
│ Opl3      │ -0.009     │ 0.163 │
│ Opmu      │ -0.023     │ 0.287 │
│ OpqMi     │ 0.037      │ 0.117 │
│ Opt       │ -1.358     │ 2.242 │
│ Opta      │ -0.042     │ 0.286 │
│ Opui      │ 0.043      │ 0.213 │
│ OtG       │ 0.075      │ 0.064 │
│ OtW       │ -0.016     │ 0.083 │
│ OtZ       │ 3.682      │ 4.387 │
│ Otap      │ 0.007      │ 0.017 │
│ Otp       │ -0.666     │ 1.784 │
└───────────┴────────────┴───────┘

           INFO     Sampling solutions ...                                                          ]8;id=519314;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/optimize/analytic.py\analytic.py]8;;\:]8;id=616290;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/optimize/analytic.py#156\156]8;;\

In [26]:
from smefit.analyze import run_report

In [27]:
run_report("./report.yaml")

[15:40:23] INFO     Analyzing : ['LHC_NLO_LIN_GLOB']                                                 ]8;id=703947;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/analyze/__init__.py\__init__.py]8;;\:]8;id=722294;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/analyze/__init__.py#34\34]8;;\

           INFO     Loading dataset : ATLAS_CMS_SSinc_RunI                                            ]8;id=401624;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py\loader.py]8;;\:]8;id=679564;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py#135\135]8;;\

           INFO     Loading dataset : ATLAS_CMS_tt_AC_8TeV                                            ]8;id=601967;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py\loader.py]8;;\:]8;id=964321;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py#135\135]8;;\

           INFO     Loading dataset : ATLAS_SSinc_RunII                                               ]8;id=105807;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py\loader.py]8;;\:]8;id=296670;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py#135\135]8;;\

           INFO     Loading dataset : ATLAS_STXS_runII_13TeV                                          ]8;id=951272;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py\loader.py]8;;\:]8;id=847789;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/loader.py#135\135]8;;\

...

Output()

/bin/sh: pandoc: command not found


/bin/sh: pandoc: command not found


[15:40:29] INFO     Plotting : chi^2 for each dataset                                                 ]8;id=453172;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/analyze/report.py\report.py]8;;\:]8;id=33742;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/analyze/report.py#170\170]8;;\

[15:40:34] INFO     Plotting : chi^2 distribution for each replica                                    ]8;id=311710;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/analyze/report.py\report.py]8;;\:]8;id=693636;file:///Users/jaco/opt/anaconda3/envs/smefit_tutorial/lib/python3.11/site-packages/smefit/analyze/report.py#175\175]8;;\